In [1]:
import os
import sys
import pickle
import csv
import copy

import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
sys.path.append("../../lib")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

../../lib/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    a

## Loading and preparing label maps

In [4]:
with open('../../temp/label_matching_coco_oi_ws.pkl', 'rb') as fin:
    labelmap = pickle.load(fin)

In [5]:
idx2mid_coco = labelmap['idx2mid_coco']
idx2mid_oi = labelmap['idx2mid_oi']
idx2mid_ws = labelmap['idx2mid_ws']
mid2idx_coco = labelmap['mid2idx_coco']
mid2idx_oi = labelmap['mid2idx_oi']
mid2idx_ws = labelmap['mid2idx_ws']
mid2name_coco = labelmap['mid2name_coco']
mid2name_oi = labelmap['mid2name_oi']
mid2name_ws = labelmap['mid2name_ws']
name2mid_coco = labelmap['name2mid_coco']
name2mid_oi = labelmap['name2mid_oi']
name2mid_ws = labelmap['name2mid_ws']
common_mids_coco = labelmap['common_mids_coco']
common_mids_ws = labelmap['common_mids_ws']


In [6]:
coco_mids = set(mid2idx_coco.keys())
oi_mids = set(mid2idx_oi.keys())
ws_mids = set(mid2idx_ws.keys())


In [7]:
with open('../../temp/all_labels_coco_oi_ws.pkl', 'rb') as fin:
    all_labels = pickle.load(fin)

In [8]:
with open('../../results/det_results_combined_oi_coco_ws_5.pkl', 'rb') as fin:
    det_results_sorted = pickle.load(fin)

## Visualizing Results

In [9]:
mid2name_extended = {}
extended_classes = set()
for key in coco_mids:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_coco[key] + f' ({abv})'
for key in oi_mids:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_oi[key] + f' ({abv})'
for key in ws_mids:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_ws[key] + f' ({abv})'
extended_classes = list(extended_classes)
mid2idx_extended = {}
for i, key in enumerate(extended_classes):
    mid2idx_extended[key] = i
category_index_extended = {mid2idx_extended[key]:{'id': mid2idx_extended[key], 'name': mid2name_extended[key]} for key in mid2name_extended}

In [10]:
det_results_forviz = copy.deepcopy(det_results_sorted)

In [11]:
for key in det_results_forviz:
    for i in range(len(det_results_forviz[key]['detection_classes'])):
        label = det_results_forviz[key]['detection_classes'][i] + '/' + det_results_forviz[key]['detection_models'][i]
        det_results_forviz[key]['detection_classes'][i] = mid2idx_extended[label]


In [12]:
print(det_results_forviz[key]['detection_classes'])

[738, 2459, 2763, 2145, 1453, 2973, 3521, 946, 1168, 1507, 1900, 133, 2630, 986, 3932, 2761, 3204, 2358, 514, 739, 2230, 432, 2022, 1710, 3301, 944, 231, 2287, 3320, 4171, 1270, 2434, 3987, 1709, 4172, 2809, 828, 530, 2223, 93, 95, 2755, 1015, 357, 642, 3882, 1530, 3436, 4147, 4038, 3525, 3436, 1616, 1645, 344, 2715, 2479, 1898, 155, 333, 3671, 2544]


In [13]:
def preproc(im):
    target_size = 400
    max_size = 1024
    im_size_min = np.min(im.shape[0:2])
    im_size_max = np.max(im.shape[0:2])
    im_scale = float(target_size) / float(im_size_min)
    # Prevent the biggest axis from being more than MAX_SIZE
    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)
    im = cv2.resize(im, None, None, fx=im_scale, fy=im_scale,
                    interpolation=cv2.INTER_LINEAR)    
    return im

In [16]:
def show(imgid, thresh):
    filename = '../../../data/seedling/' + imgid
    print(imgid)
    with open(filename, 'rb') as fin:
        #_ = fin.read(1024)
        imgbin = fin.read()
    imgbgr = cv2.imdecode(np.fromstring(imgbin, dtype='uint8'), cv2.IMREAD_COLOR)
    image_np = imgbgr[:,:,[2,1,0]]
    image_np = preproc(image_np)
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      np.asarray(det_results_forviz[imgid]['detection_boxes']),
      det_results_forviz[imgid]['detection_classes'],
      det_results_forviz[imgid]['detection_scores'],
      category_index_extended,
      instance_masks=det_results_forviz[imgid].get('detection_masks'),
      use_normalized_coordinates=True,
      min_score_thresh=thresh,
      line_thickness=2)
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    plt.show()    

In [18]:
for _ in range(10):
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, continuous_update=False), 
        imgid=fixed(np.random.choice(list(det_results_forviz.keys())))
    )
    w.children[-1].layout.height = '600px'
    display(w)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

In [19]:
for _ in range(10):
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, continuous_update=False), 
        imgid=fixed(np.random.choice(list(det_results_forviz.keys())))
    )
    w.children[-1].layout.height = '600px'
    display(w)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

In [20]:
for _ in range(10):
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, continuous_update=False), 
        imgid=fixed(np.random.choice(list(det_results_forviz.keys())))
    )
    w.children[-1].layout.height = '600px'
    display(w)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

In [21]:
for _ in range(10):
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, continuous_update=False), 
        imgid=fixed(np.random.choice(list(det_results_forviz.keys())))
    )
    w.children[-1].layout.height = '600px'
    display(w)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

In [22]:
for _ in range(10):
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, continuous_update=False), 
        imgid=fixed(np.random.choice(list(det_results_forviz.keys())))
    )
    w.children[-1].layout.height = '600px'
    display(w)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

In [23]:
for _ in range(10):
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, continuous_update=False), 
        imgid=fixed(np.random.choice(list(det_results_forviz.keys())))
    )
    w.children[-1].layout.height = '600px'
    display(w)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

In [24]:
for _ in range(10):
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, continuous_update=False), 
        imgid=fixed(np.random.choice(list(det_results_forviz.keys())))
    )
    w.children[-1].layout.height = '600px'
    display(w)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

In [25]:
for _ in range(10):
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, continuous_update=False), 
        imgid=fixed(np.random.choice(list(det_results_forviz.keys())))
    )
    w.children[-1].layout.height = '600px'
    display(w)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

In [26]:
for _ in range(10):
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, continuous_update=False), 
        imgid=fixed(np.random.choice(list(det_results_forviz.keys())))
    )
    w.children[-1].layout.height = '600px'
    display(w)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

In [27]:
for _ in range(10):
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, continuous_update=False), 
        imgid=fixed(np.random.choice(list(det_results_forviz.keys())))
    )
    w.children[-1].layout.height = '600px'
    display(w)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='thresh', max=1.0, step=0.01…